**Pseudo-labeling using MTCNN**

In [ ]:
import os, json, base64
from PIL import Image
import numpy as np
from facenet_pytorch import MTCNN
import torch

IMAGE_DIR = "data/images"      # input folder
OUT_DIR = "data/labels"   # file json save direct
SAVE_IMAGES_IN_JSON = False
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

os.makedirs(OUT_DIR, exist_ok=True)

# create detector
mtcnn = MTCNN(keep_all=True, device=DEVICE)

def pil_img_to_b64str(pil_img):
    import io
    buff = io.BytesIO()
    pil_img.save(buff, format="JPEG")
    return base64.b64encode(buff.getvalue()).decode('utf-8')

def create_labelme_json(img_path, boxes, save_path, embed_image=False):
    # boxes: list of [x1, y1, x2, y2]
    img = Image.open(img_path).convert('RGB')
    w, h = img.size
    shapes = []
    for box in boxes:
        x1, y1, x2, y2 = [float(x) for x in box]
        # LabelMe rectangle expects points: [[x1,y1],[x2,y1],[x2,y2],[x1,y2]]
        points = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
        shapes.append({
            "label": "face",
            "points": points,
            "group_id": None,
            "shape_type": "rectangle",
            "flags": {}
        })
    data = {
        "version": "4.5.7",
        "flags": {},
        "shapes": shapes,
        "imagePath": os.path.basename(img_path),
        "imageData": None,
        "imageHeight": h,
        "imageWidth": w
    }
    if embed_image:
        data["imageData"] = pil_img_to_b64str(img)
    # file json
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# Main: detect and annotations
images = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg','.jpeg','.png'))]
print(f"Found {len(images)} images in {IMAGE_DIR}. Running detection on {DEVICE} ...")

for img_name in images:
    img_path = os.path.join(IMAGE_DIR, img_name)
    try:
        # MTCNN returns boxes as Nx4 numpy array or None
        boxes, probs = mtcnn.detect(Image.open(img_path))
        boxes_list = []
        if boxes is not None:
            # optional: filter by probability threshold, e.g., probs[i] > 0.90
            for i, b in enumerate(boxes):
                if probs is None or probs[i] >= 0.6:  # threshold tuneable
                    x1, y1, x2, y2 = b.tolist()
                    # Clip to image bounds
                    img = Image.open(img_path)
                    w, h = img.size
                    x1 = max(0, min(w-1, x1))
                    x2 = max(0, min(w-1, x2))
                    y1 = max(0, min(h-1, y1))
                    y2 = max(0, min(h-1, y2))
                    boxes_list.append([x1, y1, x2, y2])
        # Save json (same name as image but .json)
        json_name = os.path.splitext(img_name)[0] + ".json"
        json_path = os.path.join(OUT_DIR, json_name)
        create_labelme_json(img_path, boxes_list, json_path, embed_image=SAVE_IMAGES_IN_JSON)
        print(f"[OK] {img_name} -> {json_name}, faces={len(boxes_list)}")
    except Exception as e:
        print(f"[ERR] {img_name}: {e}")
